<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get commits ranking from repository
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+commits+ranking+from+repository:+Error+short+description">Bug report</a>

**Tags:** #github #repos #commits #stats #naas_drivers #plotly #linechart #operations #analytics #dataframe #html

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-12 (Created: 2022-03-18)

**Description:** This notebook provides a way to view the commit rankings of a github repository.

## Input

In [1]:
import pandas as pd
import plotly.express as px
from naas_drivers import github
import naas_python as naas

### Setup variables
- `github_token`: personal token creates
- `github_url`: link to the chosen github repo

In [2]:
github_token = naas.secret.get("GITHUB_TOKEN").value
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"

## Model

### Get commits from repository url

In [3]:
commits = github.connect(github_token).repos.get_commits(repo_url)
commits

## Output

### Display result on a table

In [4]:
def get_commits(df):
    # Exclude Github commits
    df = df[(df.COMMITTER_EMAIL.str[-10:] != "github.com")]

    # Groupby and count
    df = df.groupby(["AUTHOR_NAME"], as_index=False).agg({"ID": "count"})

    # Cleaning
    df = df.rename(columns={"ID": "NB_COMMITS"})
    return df.sort_values(by="NB_COMMITS", ascending=False).reset_index(drop=True)


df = get_commits(commits)
df

### Display result on bar chart

In [5]:
def create_barchart(df, repository):
    # Get repository
    repository = repository.split("/")[-1]

    # Sort df
    df = df.sort_values(by="NB_COMMITS")

    # Calc commits
    commits = df.NB_COMMITS.sum()

    # Create fig
    fig = px.bar(
        df,
        y="AUTHOR_NAME",
        x="NB_COMMITS",
        orientation="h",
        title=f"GitHub - {repository} : Commits by user <br><span style='font-size: 13px;'>Total commits: {commits}</span>",
        text="NB_COMMITS",
        labels={"AUTHOR_NAME": "Author", "NB_COMMITS": "Nb commits"},
    )
    fig.update_traces(marker_color="black")
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=1200,
        height=800,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.show()
    return fig


fig = create_barchart(df, repo_url)